<a href="https://colab.research.google.com/github/fernando-marconi/Modelo_Preditivo_Desigualdade_Educacional/blob/main/Modelagem_Preditiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import plotly.io as pio
pio.renderers.default = "colab"

# 1. Configuração do Ambiente e Carga de Dados
drive.mount('/content/drive')
FILE_PATH = '/content/drive/MyDrive/Projeto_enem/ouro/dados_belo_horizonte_analise_final.parquet.gzip'
df = pd.read_parquet(FILE_PATH)

# 2. Definição da Variável Alvo (Target)
# Alvos definidos como 1 para o Decil 10 e 0 para os demais decis
df['target_elite'] = (df['nu_nota_ch_decil'] == 'D10').astype(int)

# 3. Seleção de Atributos e Pré-processamento
features = ['tipo_escola_agrupada', 'tp_adm_escola', 'tp_loc_escola', 'idioma_estrangeiro']
X = df[features].copy()
y = df['target_elite']

# Codificação de variáveis categóricas via One-Hot Encoding
X = pd.get_dummies(X).fillna(0)

# 4. Divisão dos Conjuntos de Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# 5. Treinamento do Modelo
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)
model.fit(X_train, y_train)

# 6. Avaliação de Performance e Matriz de Confusão
y_pred = model.predict(X_test)
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

# 7. Análise de Importância das Variáveis (Feature Importance)
importances = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

# Visualização Técnica da Importância dos Atributos
fig = px.bar(
    importances,
    x='importance',
    y='feature',
    orientation='h',
    title='Importância das Variáveis: Preditores de Desempenho de Elite',
    labels={'importance': 'Peso no Modelo', 'feature': 'Atributo'},
    template='plotly_white'
)
fig.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.99      0.68      0.80      2759
           1       0.22      0.92      0.35       269

    accuracy                           0.70      3028
   macro avg       0.60      0.80      0.58      3028
weighted avg       0.92      0.70      0.76      3028



### Validação do Modelo

In [13]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

# Cálculo da Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)

# Configuração de Rótulos e Dados
# Eixo X: O que o modelo previu | Eixo Y: A realidade dos dados
z = cm
x = ['Não Elite (D01-D09)', 'Elite (D10)']
y = ['Não Elite (D01-D09)', 'Elite (D10)']

# 3. Visualização Gráfica Anotada
fig = ff.create_annotated_heatmap(
    z, x=x, y=y,
    annotation_text=z,
    colorscale='Blues',
    showscale=True
)

fig.update_layout(
    title='Matriz de Confusão: Avaliação de Erros e Acertos',
    xaxis_title='Classe Prevista pelo Modelo',
    yaxis_title='Classe Real do Estudante',
    template='plotly_white'
)

fig.show()